In [ ]:
from __init__ import *

In [ ]:
w_, h_ = read_ws_hs(
    "../output/matrix_factorization/", SETTING["model_mode"], SETTING["model_data_"]
)

In [ ]:
distance_function = kraft.information.get_icd

In [ ]:
size = 800

layout = {"height": size, "width": size}

In [ ]:
factor_label_ = h_[0].index

factor_name = factor_label_.name

factor_label_ = factor_label_.to_numpy()

In [ ]:
w_node_x_node_distance_ = []

for i, w in enumerate(w_):

    matrix = w.to_numpy()

    node_x_node_distance = kraft.array.apply_function_on_slices_from_2_matrices(
        matrix, matrix, 1, distance_function,
    )

    w_node_x_node_distance_.append(node_x_node_distance)

    kraft.plot.plot_heat_map(
        node_x_node_distance,
        factor_label_,
        factor_label_,
        factor_name,
        factor_name,
        layout={"title": {"text": "W {} Distance".format(i)}, **layout},
    )

In [ ]:
h_node_x_node_distance_ = []

for i, h in enumerate(h_):

    matrix = h.to_numpy()

    node_x_node_distance = kraft.array.apply_function_on_slices_from_2_matrices(
        matrix, matrix, 0, distance_function,
    )

    h_node_x_node_distance_.append(node_x_node_distance)

    kraft.plot.plot_heat_map(
        node_x_node_distance,
        factor_label_,
        factor_label_,
        factor_name,
        factor_name,
        layout={"title": {"text": "H {} Distance".format(i)}, **layout},
    )

In [ ]:
node_x_node_distance = np.median(
    np.asarray(w_node_x_node_distance_ + h_node_x_node_distance_), axis=0
)

kraft.plot.plot_heat_map(
    node_x_node_distance,
    factor_label_,
    factor_label_,
    factor_name,
    factor_name,
    layout={"title": {"text": "W H Distance"}, **layout},
)

In [ ]:
node_marker_size = 24

if 1 < len(SETTING["resolution_"]):

    r_ = np.asarray(
        tuple(int(label.split(maxsplit=1)[1][0]) for label in factor_label_)
    )

    node_marker_size *= 2 ** (1 - kraft.array.normalize(r_, "0-1"))

In [ ]:
directory_path = "../output/gps_map/"

kraft.path.path(directory_path)

In [ ]:
for w_h_str in ("w", "h"):

    if w_h_str == "w":

        point_x_node = pd.concat(w_)

    else:

        point_x_node = pd.concat(h_, axis=1).T

    point_x_node **= SETTING["{}_gps_map_node_power".format(w_h_str)]

    gps_map = kraft.gps_map.GPSMap(node_x_node_distance, point_x_node)

    kraft.gps_map.write(
        "{}{}_gps_map.pickle.gz".format(directory_path, w_h_str), gps_map
    )

    gps_map.plot(
        node_trace={"marker": {"size": node_marker_size}},
        point_trace={
            "marker": {"size": SETTING["{}_gps_map_point_size".format(w_h_str)]}
        },
        file_path="{}{}_gps_map.html".format(directory_path, w_h_str),
    )